In [1]:
from quantum_circuit import *
import numpy as np
import pandas as pd 

dbg = False

act_ratios_df = pd.read_csv('../act_ratios.csv', header=None) 
act_ratios = act_ratios_df[1].values 
genes_names = act_ratios_df[0].values
theta_in = np.diag(act_ratios)
print(theta_in)
a = encoder_layer(theta_in, debug = dbg)

psi_in = np.kron([1,0],[1,0])
psi_in = np.kron(psi_in, [1,0])
psi_in = np.kron(psi_in, [1,0])
psi_in = np.kron(psi_in, [1,0])
psi_in = np.kron(psi_in, [1,0])
print(psi_in.shape, '\n', psi_in)

psi_out = np.dot(a, psi_in)
print(psi_out.shape, '\n', psi_out)

[[0.34890811 0.         0.         0.         0.         0.        ]
 [0.         0.34543392 0.         0.         0.         0.        ]
 [0.         0.         0.33047362 0.         0.         0.        ]
 [0.         0.         0.         0.31140102 0.         0.        ]
 [0.         0.         0.         0.         0.27212138 0.        ]
 [0.         0.         0.         0.         0.         0.20696256]]
(64,) 
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(64,) 
 [9.31630704e-01+0.j 1.64195868e-01+0.j 1.62527776e-01+0.j
 2.86448152e-02+0.j 1.55356175e-01+0.j 2.73808517e-02+0.j
 2.71026852e-02+0.j 4.77673064e-03+0.j 1.46239031e-01+0.j
 2.57739946e-02+0.j 2.55121524e-02+0.j 4.49640614e-03+0.j
 2.43864188e-02+0.j 4.29800048e-03+0.j 4.25433640e-03+0.j
 7.49808327e-04+0.j 1.27546359e-01+0.j 2.24794923e-02+0.j
 2.22511195e-02+0.j 3.92166324e-03+0.j 2.12692802e-02+0.j
 3.74861834e-03+0.j 3.71053552e-

In [6]:
lists = []
for i in range(6): lists.append(i)
lists[0]

0